In [6]:
import os
import glob
import netCDF4
import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt


In [7]:
input_path  = '/g/data/kl02/jss548/bonn-project-data/radolan_nc'
output_path = '/g/data/kl02/jss548/bonn-project-data/radolan_png_3time'
bad         = np.nan
min_refl    = -32.5
max_refl    = 80
x_start     = 146 #values in x dim less this this aren't present

In [8]:
def write_img(img_array, time_dt):
    #make path as needed
    image_path = '/'.join([output_path, time_dt.strftime('%Y%m%d')])
    if not os.path.isdir(image_path):
        os.mkdir(image_path)
    #build image ffn
    image_fn = 'radolan_' + time_dt.strftime('%Y%m%d_%H%M') + '.png'
    image_ffn = '/'.join([image_path, image_fn])
    #offset array using min_value
    img_array = img_array + np.abs(min_refl)
    #scale by abs maximum
    abs_max = max_refl + np.abs(min_refl)
    img_array = (img_array/abs_max)
    
    
    #export to file
    plt.imsave(image_ffn, img_array)
    count = 0
    new_run = True

In [9]:
#list files
file_list = glob.glob(input_path + '/*.nc')
for nc_file in file_list:
    #read nc file
    with netCDF4.Dataset(nc_file) as ncid:
        refl_data = np.squeeze(ncid['rx'][:, :, :]).filled(min_refl)
        time_list = np.squeeze(ncid['time'][:])
        time_units = ncid.variables['time'].units
    #export to png
    for i, time in enumerate(time_list):

        #extract refl data
        refl_img = refl_data[i, :, :]
        #crop to valid data
        refl_img_crop = refl_img[:, x_start:]

        #stack data
        if i==0:
            #init step1
            step1_img = refl_img
        elif i==1:
            #init step2
            step2_img = refl_img
        else:
            step3_img  = refl_img
            #stack steps
            refl_array = np.stack([step3_img, step2_img, step1_img], 2)
            #write to image file
            time_dt    = netCDF4.num2date(time, time_units)
            write_img(refl_array, time_dt)
            #update step1 and step2
            step2_img  = step3_img
            step1_img  = step2_img
    #compress folder
    image_path = '/'.join([output_path, time_dt.strftime('%Y%m%d')])
    cmd = ' '.join(['zip -j0r', image_path + '.zip', image_path])
    os.system(cmd)
    #update user
    print('completed:', nc_file)
    
    

completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-07.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2014-06-11.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2015-04-27.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-15.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2016-06-24.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2013-08-06.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-28.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-12.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-19.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-04.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2015-07-05.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-13.nc
completed: /g/data/kl02/jss548/bonn-project-data/radolan_nc/RX-2017-05-18.nc